# Reality Mining. Jupyter Notebook implementation


With scipy library realitymining mat file is red and saved in to data variable.
Since mat lab contains a lot of data, reading process take a lot of computer resources!

In [1]:
import scipy.io
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import math

data = scipy.io.loadmat("realitymining.mat")['s']

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
affilation = data['my_affil']
data_mat = data['data_mat']

# Restricted Boltzmann machine

### Data preprocessing.

Extract data_mat elements for each subject, categorize subjects to sloan and no sloan and save index of subjects who belongs 'sloan' category.

In [6]:
sloan_list = []
features_list = []

for i in range(len(affilation[0])):
    if len(data_mat[0][i]) > 0:
        if len(data_mat[0][i][0]) > 6 and len(affilation[0][i]) > 0:
            features_list += [data_mat[0][i]]
            if affilation[0][i][0][0][0] == 'sloan' or affilation[0][i][0][0][0] == 'sloan_2':
                sloan_list += [i]

### Exclude NaN values.
Change all NaN values to value 4 in feature_list data.

In [7]:
import math

for subject in range(len(features_list)):
    for hour in range(len(features_list[subject])):
        for element in range(len(features_list[subject][hour])):
            if math.isnan(features_list[subject][hour][element]):
                features_list[subject][hour][element] = 4

### Creating Features vector for training and testing.
<br>
__observation data__ -  value of how many days subject activity is stored. By default we have one week - 7 days.
<br>
__all_places__ - it is a list of all possible places in data_set :

 Value | Explanation
  -------------  | -------------
  0 | No signal
  1 | Home
  2 | Work
  3 | Elsewhere
  4 | Phone is off
<br>
__count__ - Current index of element position in features vector.
<br>
__features__ - 3-rd dimensional numpy N-darray for storing features for RBM training:
<br>
* 1-st dimension - Subject
* 2-nd dimension - Observation days
* 3-rd dimension - 24 hours for each place



In [10]:
#  0 – no signal, 1 – home, 2 – work, 3 – elsewhere, 4 – phone is off

observation_time = 7
all_places = [0, 1, 2, 3, 4]
count = 0

features = np.zeros((len(features_list), observation_time , 24 * len(all_places)))

for subject in range(len(features_list)):
    for week in range(observation_time):
        for number in range(len(all_places)):
            for hours in range(24):
                if features_list[subject][hours][0] == all_places[number]:
                    features[subject][week][count] = 1
                count += 1
        count = 0

# print(features)
# print(features[1][0][:24]) # No signal
# print(features[1][0][24:48]) # Home
# print(features[1][0][48:72]) # Work
# print(features[1][0][72:96]) # Elsewhere
# print(features[1][0][96:120]) # Phone is off

[[[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  1.  1.  1.]
  [ 0.  0.  0. ...,  1.  1.  1.]
  [ 0.  0.  0. ...,  1.  1.  1.]
  ..., 
  [ 0.  0.  0. ...,  1.  1.  1.]
  [ 0.  0.  0. ...,  1.  1.  1.]
  [ 0.  0.  0. ...,  1.  1.  1.]]

 ..., 
 [[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  

# Multi-layer perceptron with tensorflow

In [ ]:
for i in range(len(affilation[0])):
    if len(affilation[0][i]) > 0 and len(data_mat[0][i]) > 0:
        affilation_list += [affilation[0][i][0][0]]
        matdata_list += [data_mat[0][i].tolist()]
        if affilation_list[len(affilation_list)-1] == 'sloan' or affilation_list[len(affilation_list)-1] == 'sloan_2':
            affilation_list[len(affilation_list)-1] = [1]
#             affilation_list[len(affilation_list)-1] = [0,1]
        else:
            affilation_list[len(affilation_list)-1] = [0]

In [184]:
'''
### Classifier explanation ###
### 2 categories: sloan or no_sloan ###
### sloan = 1; no_sloan = 0 ###
'''
affilation_list = []
matdata_list = []
for i in range(len(affilation[0])):
    if len(affilation[0][i]) > 0 and len(data_mat[0][i]) > 0:
        affilation_list += [affilation[0][i][0][0]]
        matdata_list += [data_mat[0][i].tolist()]
        if affilation_list[len(affilation_list)-1] == 'sloan' or affilation_list[len(affilation_list)-1] == 'sloan_2':
            affilation_list[len(affilation_list)-1] = [1]
#             affilation_list[len(affilation_list)-1] = [0,1]
        else:
            affilation_list[len(affilation_list)-1] = [0]
#             affilation_list[len(affilation_list)-1] = [1,0]
# print(np.array(affilation_list))

In [190]:
# 1 – home, 2 – work, 3 – elsewhere, 0 – no signal, NaN – phone is off
home, work, elsewhere, no_signal, phone_off = 0, 0, 0, 0, 0
work = 0
elsewhere = 0
no_signal = 0
phone_off = 0
frequency = []
all_frequency = []
for subject in range(len(matdata_list)):
    for hours in range(24):
        for elements in range(len(matdata_list[subject][hours])):
            if matdata_list[subject][hours][elements] == 1:
                home += 1
            elif matdata_list[subject][hours][elements] == 2:
                work += 1
            elif matdata_list[subject][hours][elements] == 3:
                elsewhere += 1
            elif matdata_list[subject][hours][elements] == 0:
                no_signal += 1
            else:
                phone_off += 1
        frequency += [home/len(matdata_list[subject][hours]) if home !=0 else 0, 
                     work/len(matdata_list[subject][hours]) if  work !=0 else 0, 
                     elsewhere/len(matdata_list[subject][hours]) if elsewhere !=0 else 0, 
#                      no_signal/len(matdata_list[subject][hours]) if no_signal !=0 else 0,
                     phone_off/len(matdata_list[subject][hours]) if phone_off !=0 else 0]
        home, work, elsewhere, no_signal, phone_off = 0, 0, 0, 0, 0
    all_frequency += [frequency]
    frequency = []
features = np.array(all_frequency)
labels = np.array(affilation_list)

In [214]:
# Parameters that define the MLP
n_inputs = len(X_train[0])
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 2
X = tf.placeholder(tf.float32, shape= (None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

96


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels.ravel(), test_size=0.5, random_state=0)

In [215]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        # Number of inputs
        n_inputs = int(X.get_shape()[1])
        
        # This value is computed to randomly initialize the weights
        stddev = 2 / np.sqrt(n_inputs)
        # Weigths can be initialized in different ways
        # Here they are randomly initialized from a Normal distribution (mean=0,std as computed before)
        # Notice that weights are organized in a matrix (tensor) and its number is n_inputs*n_neurons
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        # The variable that will contain the weights is W
        W = tf.Variable(init, name="kernel")
        
        # The variable that will contain the bias is b  
        # and is initialized to zero
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        
        # As in the perceptron what the neurons do is multiply the weights by 
        # the input

        Z = tf.matmul(X, W) + b
        
        # What the activation function does is to "process" the result
        # of the multiplication of weights by inputs, and this is the output
        # of every neuron. 
    
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [216]:
# The scope name for this MLP is "dnn"
with tf.name_scope("dnn"):
    
    # The first hidden layer is defined using the RELU activation function
    # It will contain n_hidden1=300 hidden neurons and therefore output
    # 300 values    
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)

    # The second hidden layer is also defined using the RELU activation function
    # It will contain n_hidden2=100 hidden neurons and therefore output
    # 100 values    
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    
    # The output layer does not use any activation function
    # it will output n_outputs=10 values since there are 10 classes in MNIST
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

In [217]:
# scope name of the loss function is "loss"
with tf.name_scope("loss"):
    
    # The Loss function is defined. It outputs a loss value for each x
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
#     xentropy = tf.contrib.keras.backend.binary_crossentropy(output=n_inputs, target = n_inputs, from_logits=False)
    
    # The total loss is mean of the loss values 
    loss = tf.reduce_mean(xentropy, name="loss")

In [218]:
learning_rate = 0.01

with tf.name_scope("train"):
    # The plain GradientDescentOptimizer is chosen
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [219]:
with tf.name_scope("eval"):
    # A prediction is correct if it is among the k=1 most probable
    # classes predicted by the NN. Since k=1, it is only correct
    # if the prediction coincides with the true class.
    correct = tf.nn.in_top_k(logits, y, 1)
    
    # The accuracy is the mean of correct predictions
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [220]:
# Initialization of the computation graph
init = tf.global_variables_initializer()

# tensorflow allows to define a saver to store the model after learning
saver = tf.train.Saver()

# Number of epochs
n_epochs = 100

# Size of the batch used to update the gradient
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(70 // batch_size):
            
            #Function next_batch automatically select the batch
#             X_batch, y_batch = mnist.train.next_batch(batch_size)
            
            # Weights are learned using the current batch            
            sess.run(training_op, feed_dict={X: X_train, y: y_train})
            
        # Accuracies are computed in the training and validation sets    
        acc_train = accuracy.eval(feed_dict={X:  X_train, y: y_train})
        acc_val = accuracy.eval(feed_dict={X: X_test,
                                            y: y_test})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")


0 Train accuracy: 0.6 Val accuracy: 0.6
1 Train accuracy: 0.628571 Val accuracy: 0.628571
2 Train accuracy: 0.714286 Val accuracy: 0.628571
3 Train accuracy: 0.714286 Val accuracy: 0.657143
4 Train accuracy: 0.714286 Val accuracy: 0.657143
5 Train accuracy: 0.714286 Val accuracy: 0.657143
6 Train accuracy: 0.714286 Val accuracy: 0.685714
7 Train accuracy: 0.714286 Val accuracy: 0.714286
8 Train accuracy: 0.714286 Val accuracy: 0.714286
9 Train accuracy: 0.714286 Val accuracy: 0.714286
10 Train accuracy: 0.714286 Val accuracy: 0.714286
11 Train accuracy: 0.714286 Val accuracy: 0.714286
12 Train accuracy: 0.714286 Val accuracy: 0.714286
13 Train accuracy: 0.714286 Val accuracy: 0.742857
14 Train accuracy: 0.714286 Val accuracy: 0.742857
15 Train accuracy: 0.714286 Val accuracy: 0.742857
16 Train accuracy: 0.714286 Val accuracy: 0.742857
17 Train accuracy: 0.742857 Val accuracy: 0.714286
18 Train accuracy: 0.771429 Val accuracy: 0.714286
19 Train accuracy: 0.771429 Val accuracy: 0.714286


## AUTOENCODER

In [ ]:
# Since there are only three dimensions, the number of inputs is 3
n_inputs = 70

# Two neurons in the single hidden layers
n_hidden = 2  

# The number of inputs and outputs in the AE is always the same
n_outputs = n_inputs


reset_graph()
learning_rate = 0.01
X = tf.placeholder(tf.float32, shape=[None, n_inputs])

# The hidden and output layers are defined using the
# layers.dense function that does something similar to what 
# the function neuron_layer defined above does. You can see
# other parameters of this function here: https://www.tensorflow.org/api_docs/python/tf/layers/dense
hidden = tf.layers.dense(X, n_hidden, name='hidden_layer')
outputs = tf.layers.dense(hidden, n_outputs)

# The loss function is the mean-square error of the difference 
# between output and input
reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))

# Adam optimizer is used
optimizer = tf.train.AdamOptimizer(learning_rate)

# Minimization of the reconstruction loss 
training_op = optimizer.minimize(reconstruction_loss)

# The computation graph is initialized
init = tf.global_variables_initializer()

In [ ]:
# Number of iterations for the network
n_iterations = 1000
# Hidden layer 
codings = hidden

# We will keep the weights of the "hidden" layer in the variable "weights"
weights = tf.get_default_graph().get_tensor_by_name(os.path.split(hidden.name)[0] + '/kernel:0')

# The AE is run
with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        training_op.run(feed_dict={X: X_train})
        
    # We use the hidden layer as a function and it outputs
    # 2-dimensional representation of the input
    codings_val = codings.eval(feed_dict={X: X_test})

    # We print the weights learned by the network
    print(weights.eval())
 